In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

In [ ]:
reco_df = pd.DataFrame(rn.root2rec("../reco_ana.root",treename='LArbysImageTree'))
mc_df   = pd.DataFrame(rn.root2rec("../mc_ana.root",treename='mctree'))

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("/Users/vgenty/Desktop/out_0000_0099.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("empty.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
for entry in [0]:
    iom.read_entry(entry)
    larbysimg.process(iom)
    mgr=larbysimg.Manager()
    
    img_v = []

    fig,ax=plt.subplots(figsize=(20,20))
    maxx=0;minx=9999
    maxy=0;miny=9999
    plane=0

    caca_img=np.zeros((485, 485))

    for mat in mgr.InputImages():

        img_v.append(pygeo.image(mat))
        shape_img=np.where(img_v[plane]>1.5,1.0,0.0).astype(np.uint8)

        plane+=1

        nz_pixels=np.where(shape_img>0.0)
        
        maxx=np.maximum(maxx,np.max(nz_pixels[1])+100)
        minx=np.minimum(minx,np.min(nz_pixels[1])-100)

        maxy=np.maximum(maxy,np.max(nz_pixels[0])+100)
        miny=np.minimum(miny,np.min(nz_pixels[0])-100)

        caca_img+=shape_img

    caca_img=np.where(caca_img>0,1.0,0.0)

    plt.imshow(caca_img,cmap='Greys',interpolation='none')
         
    for plane in xrange(3):
        exec("x_v=reco_df.iloc[%d].circle_vtx_x%d_v"%(entry,plane))
        exec("y_v=reco_df.iloc[%d].circle_vtx_y%d_v"%(entry,plane))

        for x,y in zip(x_v,y_v):
            plt.plot(x,y,'*',color='orange',markersize=20)
        
        w_v=mc_df.iloc[entry].vtx2d_w
        t_v=mc_df.iloc[entry].vtx2d_t
        
        for w,t in zip(w_v,t_v):
            plt.plot(t,w,'*',color='yellow',markersize=20)
        
    plt.show()